In [1]:
import pandas as pd
pd.options.mode.copy_on_write = True
import matplotlib.pyplot as plt
import glob
import seaborn as sns
from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
from pathlib import Path
import shutil
import numpy as np
import os

In [15]:
#########################
#       Settings        #
#########################


year = '2025'

d_quarters = {'Q1': pd.date_range(start=pd.to_datetime("2025-02-08"),end=pd.to_datetime("2025-05-08")), 
              'Q2': pd.date_range(start=pd.to_datetime("2025-05-08"),end=pd.to_datetime("2025-08-08")), 
              'Q3': pd.date_range(start=pd.to_datetime("2025-08-08"),end=pd.to_datetime("2025-11-08")), 
              'Q4': pd.date_range(start=pd.to_datetime("2025-11-08"),end=pd.to_datetime("2026-02-08")), 
              'total': pd.date_range(start=pd.to_datetime("2025-02-08"),end=pd.to_datetime("2026-02-08")) }

date = pd.Timestamp.today().normalize()

for quarter, dates in d_quarters.items():
    if quarter == "total":
        continue
    if date in dates:
        current_quarter = quarter

time_range_previous_cycle = pd.date_range(start=pd.to_datetime("2024-02-08"),end=pd.to_datetime("2025-02-08"))

d_projects = {'BCLIMATE'  :  '2022_201', 
              'RCS'       :  '2022_202', 
              'RSDA'      :  '2022_203',
              'H-CEL'     :  '2022_204',
              'RMIB-UGENT':  '2022_205'}





#########################
#        Inputs         #
#########################

#Get the git directory
path_monitoring = f""
# load requested resources in S4C proposal

df_requested_2024 = pd.read_csv(f"{path_monitoring}requested_resources_2024.csv", delimiter=";",index_col=0)
df_transferred_2024 = pd.read_csv(f"{path_monitoring}transferred_resources_2024.csv", delimiter=";",index_col=0)

df_used_2024 = pd.read_csv(f"{path_monitoring}used_resources_2024.csv", delimiter=";",index_col=0)

# the final amount of credits available is the sum of the requested credits in the proposal and the transferred credits from the previous year, minus the use of the previous year. 

ds_left_from_2024 = df_requested_2024['total']+ df_transferred_2024['total']  - df_used_2024['total']

# redistribute what is left of 2024 evenly over quarters
df_left_from_2024 = ds_left_from_2024.to_frame(name='total')

for quarter in ['Q1', 'Q2', 'Q3', 'Q4']:
    df_left_from_2024[quarter] = df_left_from_2024['total'].div(4)

df_left_from_2024.reindex(columns=['Q1', 'Q2', 'Q3', 'Q4','total'])

# load requested resources in S4C proposal in 2025
df_requested_year = pd.read_csv(f"{path_monitoring}requested_resources_{year}.csv", delimiter=";",index_col=0)
df_transferred_year = pd.read_csv(f"{path_monitoring}transferred_resources_{year}.csv", delimiter=";",index_col=0)


df_requested = df_left_from_2024 + df_requested_year + df_transferred_year


# select one project group

#for i, group in enumerate(d_projects.keys()): 
i=0
group = list(d_projects.keys())[0]

print(group)
project = d_projects[group]

quarter_credits = df_requested.loc[group]


# read the input files
path = rf'{path_monitoring}input'
all_files = glob.glob(f'{path}/*{project}.csv')

df = pd.read_csv(all_files[0], header=1)

#########################
#     Pre-processing    #
#########################


#Get the index of the row containing project logs in resource column
index_logs = df[df['Resource'] == 'Project Logs'].index[0]
df_overview = df[:index_logs]
df_credits = df_overview[df_overview['Resource'] == 'credits']
df_credits['Total_used'] = df_credits['Current used by user'].str.replace(' Hours', '').astype(float)
df_credits['Cumulative_used_by_user'] = df_credits['Total used by user'].str.replace(' Hours', '').astype(float)
df_credits['Cumulative_total_used'] = df_credits['Total used'].str.replace(' Hours', '').astype(float)
df_credits['Date'] =  pd.to_datetime(pd.to_datetime(df_credits['Date'], format='%d/%m/%Y - %H:%M').dt.date)
df_credits = df_credits.sort_values(by='Date')

Total_credits = quarter_credits['total']

#Make a dictionary with the remaining credits per quarter by subtracting the total used from the total credits per quarter until there are no credits left
used_credits = quarter_credits.copy().loc[['Q1','Q2','Q3','Q4']]


used_credits_previouscycle = (df_credits[df_credits['Date'].isin(time_range_previous_cycle)]['Total_used']).sum()


for key in d_quarters.keys():

    df_credits_quarter = df_credits[df_credits['Date'].isin(d_quarters[key])]
    used_credits[key] = (df_credits_quarter['Total_used']).sum()
    


# save loaded credits for stacked bar plot
if i==0: 
    df_requested_credits = quarter_credits.to_frame()
    df_used_credits = used_credits.to_frame()
else: 

    df_requested_credits = df_requested_credits.merge(quarter_credits.to_frame(), left_index=True, right_index=True)
    df_used_credits = df_used_credits.merge(used_credits.to_frame(), left_index=True, right_index=True)


BCLIMATE


In [22]:
path = rf'{path_monitoring}input'
all_files = glob.glob(f'{path}/*{project}.csv')

In [26]:
all_files[-1]

'input/usage_20251217_2022_201.csv'

In [2]:
    # read the input files
    path = rf'{path_monitoring}input'
    all_files = glob.glob(f'{path}/*{project}.csv')

    df = pd.read_csv(all_files[0], header=1)

ParserError: Passed header=1 but only 1 lines in file

### Determine credits used in previous cycle

In [3]:

# determine used credits in previous cycle (2024)

for i, group in enumerate(d_projects.keys()): 
    print(group)
    project = d_projects[group]

    # read the input files
    path = rf'{path_monitoring}input'
    all_files = glob.glob(f'{path}/*{project}.csv')

    df = pd.read_csv(all_files[0], header=1)

    #Get the index of the row containing project logs in resource column
    index_logs = df[df['Resource'] == 'Project Logs'].index[0]
    df_overview = df[:index_logs]
    df_credits = df_overview[df_overview['Resource'] == 'credits']
    df_credits['Total_used'] = df_credits['Current used by user'].str.replace(' Hours', '').astype(float)
    df_credits['Cumulative_used_by_user'] = df_credits['Total used by user'].str.replace(' Hours', '').astype(float)
    df_credits['Cumulative_total_used'] = df_credits['Total used'].str.replace(' Hours', '').astype(float)
    df_credits['Date'] =  pd.to_datetime(pd.to_datetime(df_credits['Date'], format='%d/%m/%Y - %H:%M').dt.date)
    df_credits = df_credits.sort_values(by='Date')

    used_credits_previouscycle = (df_credits[df_credits['Date'].isin(time_range_previous_cycle)]['Total_used']).sum()
    df_used_2024.loc[group,"total"] = int(used_credits_previouscycle)



BCLIMATE
RCS
RSDA
H-CEL
RMIB-UGENT
